In [1]:
import ee
import os
import re
import glob
import geemap
import numpy as np
import pandas as pd
import seaborn as sns
import geopandas as gpd
from scipy.stats import norm
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

## Merge New CSV file with corrected dorminant crop with that generated from GEE using "LN_ID"

In [2]:
# Read the first CSV file
df1 = pd.read_csv('G:/001Project/Output/Sheets/original_Results/evapotranspiration/DISALEXI_ET_2018.csv')

# Read the second CSV file
df2 = pd.read_csv('G:/001Project/Output/Sheets/Irr_Croplands_csv/2018_Sheet.csv')

# Merge the two dataframes based on 'LN_ID' column
merged_df = pd.merge(df1, df2[['LN_ID', 'DOM_CROP']], on='LN_ID', how='left')

# Update the 'DOM_CROP' column in the first dataframe
df1['DOM_CROP'] = merged_df['DOM_CROP']

# Save the updated dataframe to a new CSV file
#df1.to_csv('G:/001Project/Output/Sheets/cleaned_results/evapotranspiration/trial_2016.csv', index=False)

,LN_ID,IRR_ID,CROP_ID,CROP_TYPE,FRSTDIVTXT,GEOID,COUNTY,STATE,CLIMATE,CROP_DIV,DOM_CROP
0,2401,1712729.0,121,Developed/Open Space,Section,8073,Lincoln,Colorado,BSk,3,Fallow/Idle Cropland
1,2413,2060905.0,121,Developed/Open Space,Section,8099,Prowers,Colorado,BSk,7,Alfalfa
2,2416,1331592.0,1,Corn,Section,8123,Weld,Colorado,BSk,14,Corn
3,2422,973873.0,121,Developed/Open Space,Section,8115,Sedgwick,Colorado,BSk,4,Corn
4,2424,1294969.0,1,Corn,Section,8087,Morgan,Colorado,BSk,15,Alfalfa


In [41]:
input_dir = 'G:/001Project/Output/Sheets/original_Results/evapotranspiration/'
output_dir = 'G:/001Project/Output/Sheets/original_Results/evapotranspiration/CORR_dom_crop/'

# Get a list of all the CSV files that end with "_2016"
file_list = glob.glob(input_dir + '*_2016.csv')

# Iterate through each file
for file_path in file_list:
   # Extract the base file name without the extension
    file_name = os.path.splitext(os.path.basename(file_path))[0]

    # Generate the output file path
    output_file = os.path.join(output_dir, file_name)
    
    # Generate the output file path
    output_file = os.path.join(output_dir, file_name + '.csv')
    
    print(output_file)

G:/001Project/Output/Sheets/original_Results/evapotranspiration/CORR_dom_crop/DISALEXI_ET_2016.csv
G:/001Project/Output/Sheets/original_Results/evapotranspiration/CORR_dom_crop/EEMETRIC_ET_2016.csv
G:/001Project/Output/Sheets/original_Results/evapotranspiration/CORR_dom_crop/GEESEBAL_ET_2016.csv
G:/001Project/Output/Sheets/original_Results/evapotranspiration/CORR_dom_crop/PTJPL_ET_2016.csv
G:/001Project/Output/Sheets/original_Results/evapotranspiration/CORR_dom_crop/SIMS_ET_2016.csv
G:/001Project/Output/Sheets/original_Results/evapotranspiration/CORR_dom_crop/SSEBOP_ET_2016.csv


## Add a new column called "DOM_CROP" that indicates the dominant crop in the section per area.

In [5]:
%%time

# Get a list of all the CSV files in the directory
input_dir = 'G:/001Project/Output/Sheets/original_Results/evapotranspiration/'
output_dir = 'G:/001Project/Output/Sheets/original_Results/evapotranspiration/CORR_dom_crop/'

# Create the output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Get a list of all the CSV files that end with "_2016"
file_list = glob.glob(input_dir + '*_ET_2016.csv')

# Iterate through each file
for file_path in file_list:
    # Read the CSV file
    df1 = pd.read_csv(file_path)

    # CSV file with the new information on dorminant crop
    df2 = pd.read_csv('G:/001Project/Output/Sheets/Irr_Croplands_csv/2016_Sheet.csv')

    # Merge the two dataframes based on 'LN_ID' column
    merged_df = pd.merge(df1, df2[['LN_ID', 'DOM_CROP']], on='LN_ID', how='left')

    # Update the 'DOM_CROP' column in the first dataframe
    df1['DOM_CROP'] = merged_df['DOM_CROP']
    
    # Extract the base file name without the extension
    file_name = os.path.splitext(os.path.basename(file_path))[0]

    # Generate the output file path
    output_file = os.path.join(output_dir, file_name + '.csv')

    # Save the updated dataframe to the output file
    df1.to_csv(output_file, index=False)

CPU times: total: 1min 2s
Wall time: 3min 20s


## Loop through files of common year of models clean, and combine the data

In [6]:
%%time

output_dir = 'G:/001Project/Output/Sheets/cleaned_Results/evapotranspiration/'

file_paths = [
    'G:/001Project/Output/Sheets/original_Results/evapotranspiration/CORR_dom_crop/DISALEXI_ET_2016.csv',
    'G:/001Project/Output/Sheets/original_Results/evapotranspiration/CORR_dom_crop/EEMETRIC_ET_2016.csv',
    'G:/001Project/Output/Sheets/original_Results/evapotranspiration/CORR_dom_crop/GEESEBAL_ET_2016.csv',
    'G:/001Project/Output/Sheets/original_Results/evapotranspiration/CORR_dom_crop/PTJPL_ET_2016.csv',
    'G:/001Project/Output/Sheets/original_Results/evapotranspiration/CORR_dom_crop/SIMS_ET_2016.csv',
    'G:/001Project/Output/Sheets/original_Results/evapotranspiration/CORR_dom_crop/SSEBOP_ET_2016.csv'
]

# Load the first file and keep only its details
combined_df = pd.read_csv(file_paths[0])
combined_df = combined_df.drop(columns='et_mean')

# Rename the "et_mean" column to the model name for each file and add it as a new column in the combined DataFrame
for file_path in file_paths:
    df = pd.read_csv(file_path)
    model_name = file_path.split('/')[-1].split('_')[0]
    print(f'Extracting values for ....', model_name)
    combined_df[model_name + '_ET'] = df['et_mean']

# Extract the end date using from system index, and add it to a new column
combined_df["Date"] = pd.to_datetime(combined_df["system:index"].str.split("_").str[-1], format='%Y%m%d')

# Set the 'Start Date' column as the index
combined_df.set_index('Date', inplace=True)

# drop selected columns.
combined_df = combined_df.drop(['system:index', '.geo', 'et_max', 'et_stdDev', 'FRSTDIVTXT', 'GEOID'], axis=1)

# re-arrange columns
combined_df = combined_df[['LN_ID', 'COUNTY', 'STATE',
                           'CROP_DIV', 'DOM_CROP', 'CLIMATE',	
                           'DISALEXI_ET', 'EEMETRIC_ET', 'GEESEBAL_ET', 'PTJPL_ET', 'SIMS_ET', 'SSEBOP_ET']]

# drop duplicates
# combined_df = combined_df.drop_duplicates(subset=['LN_ID', 'Date'], keep='first')

# Save the combined data to a single CSV file
combined_df.to_csv(output_dir + 'ET_Combined_2016.csv', index=True)

Extracting values for .... DISALEXI
Extracting values for .... EEMETRIC
Extracting values for .... GEESEBAL
Extracting values for .... PTJPL
Extracting values for .... SIMS
Extracting values for .... SSEBOP
CPU times: total: 37.2 s
Wall time: 55.2 s


## Loop through all the years and clean data where all model values are zero

In [7]:
%%time

output_dir = 'G:/001Project/Output/Sheets/cleaned_Results/evapotranspiration/duplicates_treated/'

file_paths = [
    'G:/001Project/Output/Sheets/cleaned_Results/evapotranspiration/ET_Combined_2016.csv',
    'G:/001Project/Output/Sheets/cleaned_Results/evapotranspiration/ET_Combined_2017.csv',
    'G:/001Project/Output/Sheets/cleaned_Results/evapotranspiration/ET_Combined_2018.csv',
    'G:/001Project/Output/Sheets/cleaned_Results/evapotranspiration/ET_Combined_2019.csv',
    'G:/001Project/Output/Sheets/cleaned_Results/evapotranspiration/ET_Combined_2020.csv',
    'G:/001Project/Output/Sheets/cleaned_Results/evapotranspiration/ET_Combined_2021.csv'
]

# Rename the "et_mean" column to the model name for each file and add it as a new column in the combined DataFrame
for file_path in file_paths:
    df = pd.read_csv(file_path)
    
    # Set rows where model values equals to 0.0 to NaN
    df.loc[(df['DISALEXI_ET'] == 0.0) & (df['EEMETRIC_ET'] == 0.0) & (df['GEESEBAL_ET'] == 0.0) & (df['PTJPL_ET'] == 0.0) & (df['SIMS_ET'] == 0.0)& (df['SSEBOP_ET'] == 0.0),
       ['DISALEXI_ET', 'EEMETRIC_ET', 'GEESEBAL_ET', 'PTJPL_ET', 'SIMS_ET', 'SSEBOP_ET']] = np.nan
    
    # Group the DataFrame by LN_ID and Date, and calculate the mean et
    df_grouped = df.groupby(['LN_ID', 'Date']).mean().reset_index()

    # Fill NaN values with 0.0 for non-numeric columns
    non_numeric_columns = ['COUNTY', 'STATE', 'CROP_DIV', 'DOM_CROP', 'CLIMATE']
    df_grouped[non_numeric_columns] = df.groupby(['LN_ID', 'Date'])[non_numeric_columns].first().reset_index(drop=True)

    combined_df = df_grouped[['Date', 'LN_ID', 'COUNTY', 'STATE',
                           'CROP_DIV', 'DOM_CROP', 'CLIMATE',	
                           'DISALEXI_ET', 'EEMETRIC_ET', 'GEESEBAL_ET', 'PTJPL_ET', 'SIMS_ET', 'SSEBOP_ET']]
    
    
        # Extract the base file name without the extension
    file_name = os.path.splitext(os.path.basename(file_path))[0]

    # Generate the output file path
    output_file = os.path.join(output_dir, file_name + '.csv')
    print(f'Processing values for ....', file_name)

    # Save the updated dataframe to the output file
    combined_df.to_csv(output_file, index=False)
    
print('Done!')

Processing values for .... ET_Combined_2016
Processing values for .... ET_Combined_2017
Processing values for .... ET_Combined_2018
Processing values for .... ET_Combined_2019
Processing values for .... ET_Combined_2020
Processing values for .... ET_Combined_2021
Done!
CPU times: total: 1min 21s
Wall time: 2min 28s


## Aggregate ET to seasonal timescales (May to September).

In [3]:
# Define the file path
file_path = r"G:\001Project\Output\Sheets\cleaned_Results\evapotranspiration\duplicates_treated\ET_Combined_2016.csv"

# Load the CSV file
data = pd.read_csv(file_path)

# Convert the "Date" column to datetime
data['Date'] = pd.to_datetime(data['Date'])

# Filter the data for May to September
seasonal_data = data[data['Date'].dt.month.isin([5, 6, 7, 8, 9])]

# Define the ET columns
et_columns = ['DISALEXI_ET', 'EEMETRIC_ET', 'GEESEBAL_ET', 'PTJPL_ET', 'SIMS_ET', 'SSEBOP_ET']

# Compute the seasonal average ET for each column
seasonal_avg_et = {col: seasonal_data[col].mean() for col in et_columns}

seasonal_avg_et


{'DISALEXI_ET': np.float64(100.2768099812758),
 'EEMETRIC_ET': np.float64(115.65483072848137),
 'GEESEBAL_ET': np.float64(94.43837628070953),
 'PTJPL_ET': np.float64(103.79434915060172),
 'SIMS_ET': np.float64(126.46239073812839),
 'SSEBOP_ET': np.float64(107.35953893876383)}

In [8]:
import pandas as pd
import os

# Define the input file pattern and output directory
input_file_template = r"G:\001Project\Output\Sheets\cleaned_Results\evapotranspiration\duplicates_treated\ET_Combined_{}.csv"
output_dir = r"G:\001Project\Output\Sheets\cleaned_Results\evapotranspiration\duplicates_treated\Seasonal_ET"

# Ensure the output directory exists
os.makedirs(output_dir, exist_ok=True)

# Define the ET columns
et_columns = ['DISALEXI_ET', 'EEMETRIC_ET', 'GEESEBAL_ET', 'PTJPL_ET', 'SIMS_ET', 'SSEBOP_ET']

# Loop through each year from 2016 to 2021
for year in range(2016, 2022):
    # Construct the input file path
    input_file = input_file_template.format(year)

    # Load the data
    data = pd.read_csv(input_file)

    # Convert the "Date" column to datetime
    data['Date'] = pd.to_datetime(data['Date'])

    # Filter the data for May to September
    seasonal_data = data[data['Date'].dt.month.isin([5, 6, 7, 8, 9])]

    # Group by 'LN_ID' and calculate the seasonal average for each ET column
    seasonal_avg = seasonal_data.groupby('LN_ID')[et_columns].mean().reset_index()

    # Save the results to a new file
    output_file = os.path.join(output_dir, f"ET_Seasonal_Averages_{year}.csv")
    seasonal_avg.to_csv(output_file, index=False)

    print(f"Processed and saved seasonal averages for {year} to: {output_file}")

print('Done!')


Processed and saved seasonal averages for 2016 to: G:\001Project\Output\Sheets\cleaned_Results\evapotranspiration\duplicates_treated\Seasonal_ET\ET_Seasonal_Averages_2016.csv
Processed and saved seasonal averages for 2017 to: G:\001Project\Output\Sheets\cleaned_Results\evapotranspiration\duplicates_treated\Seasonal_ET\ET_Seasonal_Averages_2017.csv
Processed and saved seasonal averages for 2018 to: G:\001Project\Output\Sheets\cleaned_Results\evapotranspiration\duplicates_treated\Seasonal_ET\ET_Seasonal_Averages_2018.csv
Processed and saved seasonal averages for 2019 to: G:\001Project\Output\Sheets\cleaned_Results\evapotranspiration\duplicates_treated\Seasonal_ET\ET_Seasonal_Averages_2019.csv
Processed and saved seasonal averages for 2020 to: G:\001Project\Output\Sheets\cleaned_Results\evapotranspiration\duplicates_treated\Seasonal_ET\ET_Seasonal_Averages_2020.csv
Processed and saved seasonal averages for 2021 to: G:\001Project\Output\Sheets\cleaned_Results\evapotranspiration\duplicates_

In [80]:
# Read the CSV file into a pandas DataFrame
df = pd.read_csv('G:/001Project/Output/Sheets/cleaned_Results/evapotranspiration/ET_Combined_2016.csv')

# Set rows with et_max, et_mean, and et_stdDev equal to 0.0 to NaN
df.loc[(df['DISALEXI_ET'] == 0.0) & (df['EEMETRIC_ET'] == 0.0) & (df['GEESEBAL_ET'] == 0.0) & (df['PTJPL_ET'] == 0.0) & (df['SIMS_ET'] == 0.0)& (df['SSEBOP_ET'] == 0.0),
       ['DISALEXI_ET', 'EEMETRIC_ET', 'GEESEBAL_ET', 'PTJPL_ET', 'SIMS_ET', 'SSEBOP_ET']] = np.nan

# Group the DataFrame by LN_ID and Date, and calculate the mean for et_max, et_mean, and et_stdDev
df_grouped = df.groupby(['LN_ID', 'Date']).mean().reset_index()

# Fill NaN values with 0.0 for non-numeric columns
non_numeric_columns = ['COUNTY', 'STATE', 'CROP_DIV', 'DOM_CROP', 'CLIMATE']
df_grouped[non_numeric_columns] = df.groupby(['LN_ID', 'Date'])[non_numeric_columns].first().reset_index(drop=True)

combined_df = df_grouped[['LN_ID', 'COUNTY', 'STATE',
                           'CROP_DIV', 'DOM_CROP', 'CLIMATE',	
                           'DISALEXI_ET', 'EEMETRIC_ET', 'GEESEBAL_ET', 'PTJPL_ET', 'SIMS_ET', 'SSEBOP_ET']]


## Clean Data for a single file

In [6]:
import ee
import os
import re
import numpy as np
import pandas as pd
import seaborn as sns
from scipy.stats import norm
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')


# load the CSV file using pandas
df = pd.read_csv('G:/001Project/Output/Sheets/original_results/evapotranspiration/GEESEBAL_ET_2016.csv')

# Convert the 'Start Date' column to datetime index
df["Date"] = pd.to_datetime(df["system:index"].str.split("_").str[-1], format='%Y%m%d')

# # Set the 'Start Date' column as the index
# df.set_index('Date', inplace=True)

# drop selected columns.
df = df.drop(['.geo', 'FRSTDIVTXT', 'GEOID'], axis=1)

In [8]:
selected_row = df[df['LN_ID'] == 172297]
selected_row

,system:index,CLIMATE,COUNTY,CROP_DIV,CROP_TYPE,FRSTDIVTXT,GEOID,LN_ID,STATE,et_max,et_mean,et_stdDev,.geo,Date
969720,000200000000000042f1_13s_20160101_20160131,BSk,Yuma,4,Fallow/Idle Cropland,Section,8125,172297,Colorado,22.0,5.957639,2.969430,"{""type"":""MultiPolygon"",""coordinates"":[[[[-102....",2016-01-31
969721,000200000000000042f1_13s_20160201_20160229,BSk,Yuma,4,Fallow/Idle Cropland,Section,8125,172297,Colorado,47.0,13.618377,10.531355,"{""type"":""MultiPolygon"",""coordinates"":[[[[-102....",2016-02-29
969722,000200000000000042f1_13s_20160301_20160331,BSk,Yuma,4,Fallow/Idle Cropland,Section,8125,172297,Colorado,57.0,15.390664,14.033302,"{""type"":""MultiPolygon"",""coordinates"":[[[[-102....",2016-03-31
969723,000200000000000042f1_13s_20160401_20160430,BSk,Yuma,4,Fallow/Idle Cropland,Section,8125,172297,Colorado,96.0,52.111897,19.552537,"{""type"":""MultiPolygon"",""coordinates"":[[[[-102....",2016-04-30
969724,000200000000000042f1_13s_20160501_20160531,BSk,Yuma,4,Fallow/Idle Cropland,Section,8125,172297,Colorado,76.0,32.826804,16.254507,"{""type"":""MultiPolygon"",""coordinates"":[[[[-102....",2016-05-31
969725,000200000000000042f1_13s_20160601_20160630,BSk,Yuma,4,Fallow/Idle Cropland,Section,8125,172297,Colorado,94.0,59.135629,15.212461,"{""type"":""MultiPolygon"",""coordinates"":[[[[-102....",2016-06-30
969726,000200000000000042f1_13s_20160701_20160731,BSk,Yuma,4,Fallow/Idle Cropland,Section,8125,172297,Colorado,181.0,146.927801,24.845253,"{""type"":""MultiPolygon"",""coordinates"":[[[[-102....",2016-07-31
969727,000200000000000042f1_13s_20160801_20160831,BSk,Yuma,4,Fallow/Idle Cropland,Section,8125,172297,Colorado,148.0,113.897049,22.056537,"{""type"":""MultiPolygon"",""coordinates"":[[[[-102....",2016-08-31
969728,000200000000000042f1_13s_20160901_20160930,BSk,Yuma,4,Fallow/Idle Cropland,Section,8125,172297,Colorado,82.0,57.003630,9.684884,"{""type"":""MultiPolygon"",""coordinates"":[[[[-102....",2016-09-30
969729,000200000000000042f1_13s_20161001_20161031,BSk,Yuma,4,Fallow/Idle Cropland,Section,8125,172297,Colorado,39.0,17.914094,9.567177,"{""type"":""MultiPolygon"",""coordinates"":[[[[-102....",2016-10-31


In [16]:
unique = df[df['CROP_TYPE'] == 'Corn']
unique

,system:index,CLIMATE,COUNTY,CROP_DIV,CROP_TYPE,FRSTDIVTXT,GEOID,LN_ID,STATE,et_max,et_mean,et_stdDev,.geo,Date
324,000000000000000016b1_13s_20160101_20160131,BSk,Curry,4,Corn,Section,35009,908987,New Mexico,23.0,12.441758,4.033311,"{""type"":""MultiPolygon"",""coordinates"":[[[[-103....",2016-01-31
325,000000000000000016b1_13s_20160201_20160229,BSk,Curry,4,Corn,Section,35009,908987,New Mexico,22.0,16.712821,5.177837,"{""type"":""MultiPolygon"",""coordinates"":[[[[-103....",2016-02-29
326,000000000000000016b1_13s_20160301_20160331,BSk,Curry,4,Corn,Section,35009,908987,New Mexico,46.0,27.819780,11.638230,"{""type"":""MultiPolygon"",""coordinates"":[[[[-103....",2016-03-31
327,000000000000000016b1_13s_20160401_20160430,BSk,Curry,4,Corn,Section,35009,908987,New Mexico,115.0,76.534310,14.283263,"{""type"":""MultiPolygon"",""coordinates"":[[[[-103....",2016-04-30
328,000000000000000016b1_13s_20160501_20160531,BSk,Curry,4,Corn,Section,35009,908987,New Mexico,95.0,42.949939,18.731161,"{""type"":""MultiPolygon"",""coordinates"":[[[[-103....",2016-05-31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1616035,0005000000000000074a_14s_20160801_20160831,Cfa,Washington,17,Corn,Section,20201,1076602,Kansas,152.0,123.726143,17.086391,"{""type"":""MultiPolygon"",""coordinates"":[[[[-97.2...",2016-08-31
1616036,0005000000000000074a_14s_20160901_20160930,Cfa,Washington,17,Corn,Section,20201,1076602,Kansas,117.0,82.060967,16.019185,"{""type"":""MultiPolygon"",""coordinates"":[[[[-97.2...",2016-09-30
1616037,0005000000000000074a_14s_20161001_20161031,Cfa,Washington,17,Corn,Section,20201,1076602,Kansas,68.0,47.530790,8.772029,"{""type"":""MultiPolygon"",""coordinates"":[[[[-97.2...",2016-10-31
1616038,0005000000000000074a_14s_20161101_20161130,Cfa,Washington,17,Corn,Section,20201,1076602,Kansas,23.0,10.804928,5.252311,"{""type"":""MultiPolygon"",""coordinates"":[[[[-97.2...",2016-11-30


In [9]:
value_counts = df['LN_ID'].value_counts()
value_counts.max()

36

In [ ]:
# Convert the 'Date' column to datetime if it's not already
df['Date'] = pd.to_datetime(df['Date'])

# Sort the DataFrame by 'LN_ID' and 'Date'
df.sort_values(by=['LN_ID', 'Date'], inplace=True)

# Create a mask to identify duplicates based on 'LN_ID' and 'Date'
duplicate_mask = df.duplicated(subset=['LN_ID', 'Date'], keep=False)

# Create a mask to identify rows where 'et_max', 'et_mean', and 'et_stdDev' are all 0.0
zero_mask = (df['et_max'] == 0.0) & (df['et_mean'] == 0.0) & (df['et_stdDev'] == 0.0)

# Combine the masks to drop duplicates where conditions are met
df.drop_duplicates(subset=['LN_ID', 'Date'], keep='first', inplace=True, ignore_index=True)

df = df[~(duplicate_mask & zero_mask)]

# # Define the conditions for dropping duplicates
# conditions = (df['et_max'] == 0.0) & (df['et_mean'] == 0.0) & (df['et_stdDev'] == 0.0)

# # Drop rows with duplicated dates for the 'LN_ID' column, keeping the first occurrence for rows that satisfy the conditions
# df = df[~(df.drop_duplicates(subset=['LN_ID', 'Date'], keep='first') & conditions)]

In [ ]:
# Convert the 'Date' column to datetime if it's not already
df['Date'] = pd.to_datetime(df['Date'])

# Drop rows with duplicated dates for the 'LN_ID' column
df = df.drop_duplicates(subset=['LN_ID', 'Date'], keep='first')

In [10]:
selected_rows = df[df['LN_ID'] == 2457070]
selected_rows

,system:index,CLIMATE,COUNTY,CROP_DIV,CROP_TYPE,FRSTDIVTXT,GEOID,LN_ID,STATE,et_max,et_mean,et_stdDev,.geo,Date
108084,00000000000000006876_13s_20160101_20160131,BSk,Dawson,6,Developed/Open Space,Section,48115,2457070,Texas,0.0,0.000000,0.000000,"{""type"":""GeometryCollection"",""geometries"":[{""t...",2016-01-31
108085,00000000000000006876_13s_20160201_20160229,BSk,Dawson,6,Developed/Open Space,Section,48115,2457070,Texas,0.0,0.000000,0.000000,"{""type"":""GeometryCollection"",""geometries"":[{""t...",2016-02-29
108086,00000000000000006876_13s_20160301_20160331,BSk,Dawson,6,Developed/Open Space,Section,48115,2457070,Texas,0.0,0.000000,0.000000,"{""type"":""GeometryCollection"",""geometries"":[{""t...",2016-03-31
108087,00000000000000006876_13s_20160401_20160430,BSk,Dawson,6,Developed/Open Space,Section,48115,2457070,Texas,0.0,0.000000,0.000000,"{""type"":""GeometryCollection"",""geometries"":[{""t...",2016-04-30
108088,00000000000000006876_13s_20160501_20160531,BSk,Dawson,6,Developed/Open Space,Section,48115,2457070,Texas,0.0,0.000000,0.000000,"{""type"":""GeometryCollection"",""geometries"":[{""t...",2016-05-31
108089,00000000000000006876_13s_20160601_20160630,BSk,Dawson,6,Developed/Open Space,Section,48115,2457070,Texas,0.0,0.000000,0.000000,"{""type"":""GeometryCollection"",""geometries"":[{""t...",2016-06-30
108090,00000000000000006876_13s_20160701_20160731,BSk,Dawson,6,Developed/Open Space,Section,48115,2457070,Texas,0.0,0.000000,0.000000,"{""type"":""GeometryCollection"",""geometries"":[{""t...",2016-07-31
108091,00000000000000006876_13s_20160801_20160831,BSk,Dawson,6,Developed/Open Space,Section,48115,2457070,Texas,0.0,0.000000,0.000000,"{""type"":""GeometryCollection"",""geometries"":[{""t...",2016-08-31
108092,00000000000000006876_13s_20160901_20160930,BSk,Dawson,6,Developed/Open Space,Section,48115,2457070,Texas,0.0,0.000000,0.000000,"{""type"":""GeometryCollection"",""geometries"":[{""t...",2016-09-30
108093,00000000000000006876_13s_20161001_20161031,BSk,Dawson,6,Developed/Open Space,Section,48115,2457070,Texas,0.0,0.000000,0.000000,"{""type"":""GeometryCollection"",""geometries"":[{""t...",2016-10-31
